In [1]:
import sys, os, itertools
import re, nltk
from nltk import pos_tag
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import sklearn, gensim
from sklearn.decomposition import PCA
from gensim.corpora import Dictionary
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
os.chdir(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/data/')
from utils import preprocess_for_bow
from models.lda import LDAwrappers
from models.hdp import HDPwrapper
from models.gsdmm import MovieGroupProcessWrapper
from models.lftm import LFTMwrapper
from models.bertopic import BERTopicWrapper
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
data = preprocess_for_bow('data.csv', preprocessing=False)

In [3]:
data.keys()

dict_keys(['text', 'ids'])

In [4]:
bertopic=BERTopicWrapper(data['text'], data['ids'])

In [5]:
bertopic.model.fit(data['text'])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [6]:
len(bertopic.model.topics_) #list of topic indexes
#bertopic.model.topic_representations_
#bertopic.model.probabilities_ #probs assigned topics per documents
#bertopic.model.topic_aspects_
#bertopic.model.approximate_distribution(data['text'])[0].shape

30192

## TOPIC representations

In [82]:
bertopic.model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,12061,-1_covid_covid19_biden_president,"[covid, covid19, biden, president, virus, peop...",[China has closed down its last coronavirus ho...
1,0,596,0_masks_mask_face_wear,"[masks, mask, face, wear, wearing, wearing mas...","[Can wearing masks be dangerous, Face Masks Do..."
2,1,549,1_tweet_misleading_hit_open,"[tweet, misleading, hit, open, context, add, t...",[If you want to add helpful context to any Twe...
3,2,535,2_church_holy_god_council,"[church, holy, god, council, abuse, religious,...","[And at a later date, he assured us that that ..."
4,3,370,3_hoax_deep state_deep_nwo,"[hoax, deep state, deep, nwo, covid hoax, covi...",[#COVID19 is the tool public enemy no. 1 the h...
...,...,...,...,...,...
653,652,6,652_australia_covid19 vaccination_reject_austr...,"[australia, covid19 vaccination, reject, austr...",[The Western Australian state government descr...
654,653,6,653_executed_jail_facing_committing,"[executed, jail, facing, committing, fewer, fe...",[The Trump administration executed more people...
655,654,6,654_peoples_foreign_approach_committing,"[peoples, foreign, approach, committing, dream...",[#China is a country that is always committed ...
656,655,6,655_boycott_chinese_goods_52,"[boycott, chinese, goods, 52, shows chinese, c...","[Govt, WHO issued advisory to not purchase Chi..."


In [21]:
bertopic.model.c_tf_idf_.toarray().shape #[:,1].toarray().argmax()

(658, 42630)

## computing dot-product and cosine similarity of topic representations with respect to label -1 (most popular cluster, essentially composed of docs that do not fit well any clusters)

In [54]:
ref_vector = bertopic.model.c_tf_idf_.toarray()[0]
topic_to_eval=[bertopic.model.c_tf_idf_.toarray()[i] for i in range(len(bertopic.model.c_tf_idf_.toarray()))]
dot_products = np.dot(topic_to_eval, ref_vector)

reference_norm = np.linalg.norm(ref_vector)
vector_norms = np.linalg.norm(topic_to_eval, axis=1)
cosine_similarities = dot_products / (reference_norm * vector_norms)

In [55]:
results = [(i, dot_product, cosine_similarity) for i, (dot_product, cosine_similarity) in enumerate(zip(dot_products, cosine_similarities))]

# Sort the results by dot product or cosine similarity, depending on your preference
results_sorted_by_dot_product = sorted(results, key=lambda x: x[1], reverse=True)
results_sorted_by_cosine_similarity = sorted(results, key=lambda x: x[2], reverse=True)

### top similarity

In [68]:
results_sorted_by_cosine_similarity[:20]

[(0, 0.000843716914209178, 1.0),
 (4, 0.0008174226399800058, 0.5679582606776222),
 (23, 0.0008209675799919817, 0.46205681925996656),
 (3, 0.0008254824489434142, 0.41164446490793055),
 (41, 0.0008072429358391409, 0.37810818810815194),
 (1, 0.0007276457863104305, 0.33870153588832425),
 (128, 0.0008076020873097997, 0.3307343137367248),
 (7, 0.0007153253066766515, 0.32661334712107437),
 (56, 0.0008171933665713882, 0.3189743156757132),
 (16, 0.0008106477285220252, 0.30703974899713227),
 (169, 0.0007974544655734244, 0.30236896964770216),
 (26, 0.0008026104149230939, 0.2984790328719456),
 (8, 0.0007830676072873902, 0.29753476637898096),
 (108, 0.0007741407328216784, 0.29734024988876184),
 (130, 0.0008395067407591023, 0.2929236191711839),
 (39, 0.0008028642617115262, 0.2912797567802683),
 (25, 0.0008078933647774043, 0.2861430277511942),
 (10, 0.0007988572339182193, 0.27630794810356296),
 (33, 0.0008524732738811168, 0.2715586758342532),
 (11, 0.0008032495089316603, 0.26994156834547134)]

In [61]:
bertopic.model.get_topic_info().iloc[[4,23,3,41,1,128,7,56,16,169,26,8,108]]

,Topic,Count,Name,Representation,Representative_Docs
4,3,370,3_hoax_deep state_deep_nwo,"[hoax, deep state, deep, nwo, covid hoax, covi...",[#COVID19 is the tool public enemy no. 1 the h...
23,22,111,22_pandemic_plandemic_population control_control,"[pandemic, plandemic, population control, cont...",[This isn’t a pandemic! It’s a plandemic and w...
3,2,535,2_church_holy_god_council,"[church, holy, god, council, abuse, religious,...","[And at a later date, he assured us that that ..."
41,40,68,40_microchip_microchips_covid vaccine_vaccine,"[microchip, microchips, covid vaccine, vaccine...",[So what’s the protocol when you find out your...
1,0,596,0_masks_mask_face_wear,"[masks, mask, face, wear, wearing, wearing mas...","[Can wearing masks be dangerous, Face Masks Do..."
128,127,33,127_china_bioweapon_chinese_wuhan,"[china, bioweapon, chinese, wuhan, originated,...",[You totally misunderstood what she was saying...
7,6,264,6_italy_bodies_victims_italian,"[italy, bodies, victims, italian, patient, pat...","[Mass dead bodies of coronavirus victims, This..."
56,55,58,55_qanon_conspiracy_cult_conspiracy theory,"[qanon, conspiracy, cult, conspiracy theory, t...",[I just hope the Biden administration takes ve...
16,15,139,15_gates_wants_foundation_depopulation,"[gates, wants, foundation, depopulation, vacci...",[Bill Gates created the new coronavirus and a ...
169,168,26,168_biden_economy_debate_america,"[biden, economy, debate, america, hidden, qano...",[Biden is fulfilling his America last agenda.⬇...


### bottom similarity docs

In [71]:
results_sorted_by_cosine_similarity[-20:]

[(398, 0.0007556373987942029, 0.053725848504703425),
 (534, 0.000879326133568352, 0.05306817750995376),
 (363, 0.0008717249324719934, 0.053050763073307514),
 (585, 0.0007235171167017703, 0.052586928178368096),
 (643, 0.0007594053039702482, 0.052408663235525385),
 (411, 0.0005208117380437931, 0.0516900348892993),
 (287, 0.0005486969593291723, 0.05106922910810369),
 (521, 0.000832816326687511, 0.050985702481521136),
 (446, 0.0006663066506357508, 0.05093057075786086),
 (559, 0.0008977131933935585, 0.05025821812535979),
 (116, 0.0006818373432709341, 0.04980355275763158),
 (578, 0.000945818607170907, 0.04871492807078553),
 (518, 0.0007645885410815349, 0.04791697752231553),
 (489, 0.0005782933535065727, 0.03964159071820222),
 (583, 0.0008561668251227939, 0.03953814417443294),
 (551, 0.0004848303650232284, 0.03804609007428984),
 (377, 0.0006961019117969691, 0.037103246814631466),
 (511, 0.0006837509388547595, 0.0360830606671647),
 (503, 0.0006912678754198258, 0.032367922662399816),
 (2, 9.278

In [72]:
bertopic.model.get_topic_info().iloc[[398, 534, 363, 585, 643, 411]]

,Topic,Count,Name,Representation,Representative_Docs
398,397,12,397_netflix_movie_season_wish,"[netflix, movie, season, wish, christian, remo...","[Netflix removes all Christian movies…, The Ho..."
534,533,8,533_permission_article_posted_project,"[permission, article, posted, project, locked,...","[Article posted with permission from SHTFPlan,..."
363,362,13,362_net_worth_left_family,"[net, worth, left, family, final, circulating,...",[Rush Limbaugh's final net worth was revealed ...
585,584,7,584_football_ground_worlds_light,"[football, ground, worlds, light, ready, middl...",[Photo shows a football pitch in the middle of...
643,642,6,642_tank_backed_therapy_smoking,"[tank, backed, therapy, smoking, injected, end...",[Tom Selleck is a spokesperson for CBD gummies...
411,410,11,410_se_el_da_dan,"[se, el, da, dan, na, film, una, reset, en, del]",[Francuski film Hold-up dokazuje da je pandemi...


In [73]:
results_sorted_by_cosine_similarity[100:120]

[(35, 0.0008179745548994204, 0.17665575694948532),
 (460, 0.0007943473355710971, 0.1762888364867454),
 (229, 0.0008597319190668643, 0.17548281091257528),
 (214, 0.0008325835543988467, 0.17484943466506767),
 (151, 0.0007761221898717529, 0.17353980263574278),
 (272, 0.0007241354872267353, 0.17329726138894683),
 (606, 0.0008312105306366894, 0.17299887976126568),
 (123, 0.0007842077835071984, 0.17238316752892185),
 (312, 0.0008263518560606075, 0.1723656846073271),
 (252, 0.0008338509360315924, 0.17169502278268362),
 (118, 0.0008357396323427226, 0.17168900437670057),
 (181, 0.0008079805974063628, 0.17160505769604267),
 (621, 0.000838291180602718, 0.17126541346398205),
 (145, 0.0007161348210620424, 0.17109740243605898),
 (469, 0.000835281464309449, 0.17084381655553496),
 (117, 0.0008114205035581329, 0.17025494977858485),
 (28, 0.0007573781333422025, 0.16960239202197677),
 (295, 0.0008932306265802378, 0.16876148513658537),
 (338, 0.0008496930272358049, 0.16872748230210358),
 (555, 0.000844040

In [74]:
bertopic.model.get_topic_info().iloc[[35, 460, 229, 214, 151]]

,Topic,Count,Name,Representation,Representative_Docs
35,34,79,34_antifa_portland_capitol_trump supporters,"[antifa, portland, capitol, trump supporters, ...",[The FBI has said that most Antifa members wor...
460,459,10,459_starts_hollywood_democratic presidential_s...,"[starts, hollywood, democratic presidential, s...",[Actor/director Clint Eastwood wrote an opinio...
229,228,20,228_beast_mark_antichrist_covid vaccine,"[beast, mark, antichrist, covid vaccine, comin...",[Remember: Revelation 13 is clear on “The Mark...
214,213,22,213_text_sex_messages_global,"[text, sex, messages, global, available, store...","[You got us, there’s no Eggplant condom! But w..."
151,150,28,150_soldiers_massacre_victory_army,"[soldiers, massacre, victory, army, lets, dama...","[⭕️ #Ethiopia .. #TigrayGenocide, Photo shows ..."


In [75]:
results_sorted_by_cosine_similarity[45:65]

[(89, 0.0007650083176313123, 0.22010946617150992),
 (315, 0.0008201900147784711, 0.219653065830668),
 (48, 0.0007858672333410589, 0.21934497356114455),
 (86, 0.0007587168671165563, 0.21891821265625952),
 (173, 0.0008712318015804349, 0.21844635724725806),
 (19, 0.0006742897557780703, 0.21529441476117042),
 (47, 0.0008240723493345005, 0.21514744686374612),
 (102, 0.0008354098037920428, 0.21429663278346006),
 (120, 0.0007042623487511187, 0.21327982765361075),
 (17, 0.000827836160910158, 0.21271936221769436),
 (213, 0.0007829958927455547, 0.21253626923392538),
 (40, 0.0008994306144952059, 0.21203618691919404),
 (66, 0.0007674309902090755, 0.2117235857156604),
 (57, 0.0007910819698765618, 0.21101193895891546),
 (149, 0.0008176471259366438, 0.20911306913446243),
 (158, 0.0008881467885804302, 0.20871360142623155),
 (442, 0.0008375817314716711, 0.20680571106424367),
 (90, 0.0008136641406981339, 0.20629054181852655),
 (82, 0.000684117312022006, 0.20586894676022363),
 (84, 0.0007618027662385882,

In [76]:
bertopic.model.get_topic_info().iloc[[89, 315, 48, 86, 173, 19, 47]]

,Topic,Count,Name,Representation,Representative_Docs
89,88,42,88_immunity_herd_covid19 vaccines_data,"[immunity, herd, covid19 vaccines, data, vacci...",[These numbers reflect the latest data availab...
315,314,14,314_chinas_asian_fault_china,"[chinas, asian, fault, china, constitution, co...",[Hilarious... So much for China being a Hostil...
48,47,63,47_georgia_voting_suppression_voter,"[georgia, voting, suppression, voter, brian, l...",[Georgia Democratic Candidate for Governor Sta...
86,85,43,85_endorsed_joe biden_joe_presidential,"[endorsed, joe biden, joe, presidential, nomin...",[The AARP endorsed Democratic presidential nom...
173,172,25,172_homes_patients_governors_sent,"[homes, patients, governors, sent, andrew, que...",[New York Gov. Andrew Cuomo ordered thousands ...
19,18,122,18_water_delhi_bridge_hyderabad,"[water, delhi, bridge, hyderabad, recent, heav...",[Video shows flooding in south China in June 2...
47,46,64,46_congress_green_leader_performed,"[congress, green, leader, performed, indias, v...",[Rahul and Priyanka Gandhi laughing on their w...


In [79]:
results_sorted_by_cosine_similarity[25:40]

[(27, 0.0007253528177224511, 0.2571937343794789),
 (58, 0.0008547957270441378, 0.2488022484841877),
 (13, 0.0008834116718997588, 0.245545751524216),
 (77, 0.0008186887927594052, 0.24521512432565995),
 (62, 0.0007854407220310092, 0.24512031476055604),
 (65, 0.0008101493755424306, 0.24306839465405616),
 (72, 0.000823371521860659, 0.2396201785435091),
 (70, 0.0007936171552814794, 0.23942305643145756),
 (99, 0.0008313347020407105, 0.2367912631716071),
 (34, 0.0007922526087130221, 0.23617113650074395),
 (30, 0.0008369536859258039, 0.2350641887474674),
 (12, 0.0007496977449899743, 0.2329515791073379),
 (6, 0.0007298961473291683, 0.23267056928844954),
 (37, 0.0008330439418967323, 0.23147467797852359),
 (138, 0.000874787351433492, 0.2308422740834824)]

In [80]:
bertopic.model.get_topic_info().iloc[[27, 58, 13, 77, 62, 65]]

,Topic,Count,Name,Representation,Representative_Docs
27,26,99,26_stolen_election_fraud_trump won,"[stolen, election, fraud, trump won, won, stol...",[The 2020 election was a fraud!!! Trump won by...
58,57,56,57_babies_birth_mothers_planned,"[babies, birth, mothers, planned, certificate,...",[Our taxpayer dollars should NEVER go to fundi...
13,12,169,12_aoc_email_guys_best,"[aoc, email, guys, best, amazing, pass, mouth,...",[@caeser_pounce @MattyThrice @neontaster If @M...
77,76,46,76_senators_senate_vote_act,"[senators, senate, vote, act, voting, ensure, ...","[Joe Biden said, """"With regard to the filibust..."
62,61,55,61_predicted_cold_new coronavirus_novel,"[predicted, cold, new coronavirus, novel, nove...","[Novel coronavirus is just the common cold, Au..."
65,64,51,64_impeachment_senators_senate_office,"[impeachment, senators, senate, office, trial,...",[Four facts will never change: 1. This #impeac...


In [81]:
bertopic.model.get_topic_info().iloc[[99, 34, 30, 12, 6, 37, 138]]

,Topic,Count,Name,Representation,Representative_Docs
99,98,39,98_shooter_suspect_cop_white,"[shooter, suspect, cop, white, shooting, kille...","[My apologies to the many, many, many angry wh..."
34,33,81,33_australian_australia_burning_california,"[australian, australia, burning, california, s...",[These photos show a dog which saved a koala f...
30,29,95,29_islamic_islam_religion_muslims,"[islamic, islam, religion, muslims, false flag...",[Muslims who want to live under Islamic Sharia...
12,11,170,11_philippines_senator_vice president_vice,"[philippines, senator, vice president, vice, 2...","[In November 2020, former senator Bongbong Mar..."
6,5,268,5_ballots_ballot_voters_voter,"[ballots, ballot, voters, voter, registered, v...",[Mail-In Ballots Voter Fraud Captured on Video...
37,36,77,36_climate_climate change_change_ice,"[climate, climate change, change, ice, global,...","[Patrick Moore, co-founder of Greenpeace: “The..."
138,137,31,137_press_secretary_white house_white,"[press, secretary, white house, white, white s...",[Just when you thought story cannot get any mo...


## THRESHOLD of 0.25/0.24 seems appropriate 

In [51]:
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1, 2), min_df=10) #can override preprocessor
                                            #strip_accents
                                            #min_df (min occurence over the whole corpus ?)
bertopic.model.update_topics(data['text'], vectorizer_model=vectorizer_model)

In [52]:
bertopic.model.c_tf_idf_.toarray().shape

(658, 3096)

In [25]:
bertopic.model.get_topic_info()[:20]

,Topic,Count,Name,Representation,Representative_Docs
0,-1,12061,-1_for_of_is_the,"[for, of, is, the, are, covid19, was, in, with...",[China has closed down its last coronavirus ho...
1,0,596,0_masks_mask_face_wearing,"[masks, mask, face, wearing, wear, ineffective...","[Can wearing masks be dangerous, Face Masks Do..."
2,1,549,1_tweet_contribute_menu_helpful,"[tweet, contribute, menu, helpful, might, misl...",[If you want to add helpful context to any Twe...
3,2,535,2_pope_church_catholic_bishops,"[pope, church, catholic, bishops, cardinal, fr...","[And at a later date, he assured us that that ..."
4,3,370,3_hoax_deep_covid_nwo,"[hoax, deep, covid, nwo, sheeple, state, world...",[#COVID19 is the tool public enemy no. 1 the h...
5,4,332,4_de_en_la_un,"[de, en, la, un, del, el, que, foto, di, con]",[En @UInvestiga obtuvimos una copia del expedi...
6,5,268,5_ballots_mailin_ballot_voters,"[ballots, mailin, ballot, voters, voter, votes...",[Mail-In Ballots Voter Fraud Captured on Video...
7,6,264,6_italy_bodies_victims_italian,"[italy, bodies, victims, italian, patients, pa...","[Mass dead bodies of coronavirus victims, This..."
8,7,234,7_israeli_israel_gaza_palestinians,"[israeli, israel, gaza, palestinians, palestin...",[Israel-Palestinians Latest: • Fighting bears ...
9,8,212,8_mcdonalds_anniversary_pizza_coupon,"[mcdonalds, anniversary, pizza, coupon, offeri...",[Pizza Hut is offering four free large pizzas ...


In [31]:
bertopic.model.get_topic_info()[20:]

,Topic,Count,Name,Representation,Representative_Docs
20,19,120,19_border_wall_caravan_migrants,"[border, wall, caravan, migrants, patrol, paso...","[Trump's border wall is 'almost complete', Bor..."
21,20,118,20_farmers_tractor_protesting_protest,"[farmers, tractor, protesting, protest, farm, ...",[Photo shows farmers protesting in Delhi again...
22,21,115,21_modi_narendra_pm_modis,"[modi, narendra, pm, modis, ambani, prime, min...",[Image shows Narendra Modi's statue has been i...
23,22,111,22_pandemic_plandemic_control_population,"[pandemic, plandemic, control, population, wak...",[This isn’t a pandemic! It’s a plandemic and w...
24,23,111,23_cure_water_salt_drinking,"[cure, water, salt, drinking, lemon, tea, ging...",[Facebook posts claims gargling warm salt wate...
...,...,...,...,...,...
653,652,6,652_australia_hivpositive_poisons_suspends,"[australia, hivpositive, poisons, suspends, 81...",[The Western Australian state government descr...
654,653,6,653_jail_executed_offences_379998,"[jail, executed, offences, 379998, prisons, 19...",[The Trump administration executed more people...
655,654,6,654_humanrights_70plus_peoplecentered_peaceful...,"[humanrights, 70plus, peoplecentered, peaceful...",[#China is a country that is always committed ...
656,655,6,655_diwali_apps_advisory_boycott,"[diwali, apps, advisory, boycott, chinese, itb...","[Govt, WHO issued advisory to not purchase Chi..."


In [17]:
#bertopic.model.get_document_info(data['text'])

In [18]:
#bertopic.model.get_representative_docs()

In [19]:
#bertopic.model.generate_topic_labels()

In [ ]:
bertopic.model.merge_topics(docs, topics_to_merge)

In [ ]:
bertopic.model.reduce_outliers(docs, topics)